In [1]:
import torch
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors

%matplotlib widget 

Environment closely follows OpenAI gym API. Currently can not be invoked with ```gym.make("env_id")```, though it should be easy to do.

In [2]:
import env
checkers_env = env.Env()

In [3]:
""" Various plotting helper methods """

def show_state(state):
    plt.imshow(state)
    plt.show()

def show_board(board):
   cmap = colors.ListedColormap(['white', 'red'])
   bounds=[0,0.5,18]
   norm = colors.BoundaryNorm(bounds, cmap.N)
   # plt.figure(figsize=(4,4))
   plt.imshow(board, cmap=cmap, norm=norm, interpolation='none')
   plt.xticks(np.arange(0.5,8.5), [])
   plt.yticks(np.arange(0.5,8.5), [])

   plt.grid()

def do_the_flip(arr):
    return np.flip(np.flip(np.flip(arr, 0), 0), 1)

def show_trajectory_item(trajectory, index):
    board_i = monte_carlo_tree.state_to_board(trajectory[index].original_state)
    show_board(do_the_flip(board_i[0] + board_i[1]))

## Random Play Tree

This is very basic algorithm that plays the game by making random moves. Sometimes it reaches the end goal, but overall it supper inneficient.

In [4]:
import monte_carlo_tree


def random_play():
    '''
    Play a game using random tree strategy
    '''
        
    tree = monte_carlo_tree.RandomPlayTree(checkers_env, 8)
    
    root_node = tree.root_node
    return tree.simulate(root_node)
    

def build_stats(playfunc, n_games=100):
    '''
    Play a number of random games and display result
    '''

    black_wins = 0
    white_wins = 0
    draws = 0
    moves = []
    
    for _ in range(n_games):
        m = playfunc()
        if m.reward > 0:
            black_wins += 1
        elif m.reward < 0:
            white_wins += 1
        elif m.reward == 0:
            draws += 1
       
        moves.append(m.depth())
    
    print("Blacks: ", black_wins, "Whites: ", white_wins, "Draws: ", draws, "Moves mean:", np.mean(moves))

In [5]:
# build_stats(random_play, 100)

Blacks:  3 Whites:  4 Draws:  93 Moves mean: 51.39


## Monte Carlo Search Tree

In [6]:
from monte_carlo_tree import MonteCarloPlayTree


def mtsc_play():
    '''
    Play a game using MonteCarloSearchTree
    '''

    mcst = MonteCarloPlayTree(checkers_env, 8)
    
    root_node = mcst.root_node
    terminal_node = mcst.simulate(root_node)
    
    return terminal_node


In [7]:
build_stats(mtsc_play, 100)

KeyError: 0

## Guided Tree Search

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

BOARD_SIZE = 8

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ActorCritic(nn.Module):

    def __init__(self, board_size=BOARD_SIZE):
        super(ActorCritic, self).__init__()
        
        self.board_size = BOARD_SIZE

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Policy head
        self.act_conv1 = nn.Conv2d(128, 4, kernel_size=1)
        self.act_fc1 = nn.Linear(4*(self.board_size**2), self.board_size**4)
        
        # Critic head
        self.val_conv1 = nn.Conv2d(128, 2, kernel_size=1)
        self.val_fc1 = nn.Linear(2*self.board_size**2, 64)
        self.val_fc2 = nn.Linear(64, 1)
        
    def forward(self, x):

        x = x.unsqueeze(1)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        
        # Actor head
        y_actions = torch.relu(self.act_conv1(x)).view(-1, 4*self.board_size**2)
        y_actions = self.act_fc1(y_actions)
        y_actions = torch.exp(y_actions-torch.max(y_actions))
        y_actions = (y_actions/torch.sum(y_actions))
        
        # Critic head
        y_value = torch.relu(self.val_conv1(x))
        y_value = y_value.view(-1, 2*(self.board_size**2))
        y_value = F.relu(self.val_fc1(y_value))
        y_value = torch.tanh(self.val_fc2(y_value))

        return y_actions, y_value

actor_critic_network = ActorCritic().to(device)

In [ ]:
tree = monte_carlo_tree.GuidedMonteCarloPlayTree(checkers_env, BOARD_SIZE, actor_critic_network, device)

In [ ]:
tree.train(100)

In [ ]:
torch.save(actor_critic_network.state_dict(), "./../assets/actor_critic.pt")

In [ ]:
def gmcts_play():
    '''
    Play a game using random tree strategy
    '''
        
    tree = monte_carlo_tree.GuidedMonteCarloPlayTree(checkers_env, BOARD_SIZE, actor_critic_network, device)
    
    root_node = tree.root_node
    return tree.simulate(root_node)

In [ ]:
build_stats(gmcts_play, 100)

In [ ]:
import torch
import torchvision

model = torchvision.models.resnet18(pretrained=True)
model.eval()
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save("model.pt")